In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
from scipy.io import wavfile
import glob
from nnAudio import Spectrogram
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
import time
import tqdm
import pickle
from pathlib import Path
import os
import numpy as np

p = os.path.join('../../../../pytorch_musicnet/data/', 'train_data', '*.wav')

In [2]:
from nvidia.dali.pipeline import Pipeline
from nvidia.dali.plugin.pytorch import DALIGenericIterator
import nvidia.dali.ops as ops
import types

In [4]:
class MusicNet(Dataset):
    """Face Landmarks dataset."""

    def __init__(self, transform=None):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
#         print(Path(__file__).parent)
        self.file_list = glob.glob(p)
        
    def __len__(self):
        return len(self.file_list)  
    
    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
        audio_name = self.file_list[idx]
        sr, wav = wavfile.read(audio_name)

        return wav

In [5]:
dataset = MusicNet()
dataset = DataLoader(dataset, shuffle=False, num_workers=8)
result = {}

In [5]:
class SpectrogramPipeline(Pipeline):
    def __init__(self, device, batch_size, num_threads=1, device_id=0):
        super(SpectrogramPipeline, self).__init__(batch_size, num_threads, device_id)
        self.device = device
        self.batch_data = []
        
        for _ in range(batch_size):
            y, sr = librosa.load(librosa.util.example_audio_file())
            self.batch_data.append(np.array(y, dtype=np.float32))

        self.external_source = ops.ExternalSource()
        self.spectrogram = ops.Spectrogram(device=self.device,
                                           nfft=nfft,
                                           window_length=window_length,
                                           window_step=window_step)

    def define_graph(self):
        self.data = self.external_source()
        out = self.data.gpu() if self.device == 'gpu' else self.data
        out = self.spectrogram(out)
        return out

    def iter_setup(self):
        self.feed_input(self.data, self.batch_data)

In [7]:
class ExternalInputIterator(object):
    def __init__(self, batch_size):
        self.batch_size = batch_size
        self.files = glob.glob(p)

    def __iter__(self):
        self.i = 0
        self.n = len(self.files)
        return self

    def __next__(self):
        batch = []
        for _ in range(self.batch_size):
            audio_name = self.files[self.i]
            sr, wav = wavfile.read(audio_name)            
            
            batch.append([wav])
            self.i = (self.i + 1) % self.n
            print(f'batch shape = {wav.shape}')
        return batch
    
class ExternalSourcePipeline(Pipeline):
    def __init__(self, batch_size, num_threads, device_id):
        super(ExternalSourcePipeline, self).__init__(batch_size,
                                      num_threads,
                                      device_id,
                                      seed=12)

        self.source = ops.ExternalSource(source = eii, num_outputs = 1)
        print('pass')

    def define_graph(self):
        jpegs = self.source()
        print('pass 2')
        return jpegs
    

In [8]:
batch_size = 1
eii = ExternalInputIterator(batch_size)
iterator = iter(eii)

In [9]:
pipe = ExternalSourcePipeline(batch_size=batch_size, num_threads=8, device_id = 0)
pipe.build()

pass
pass 2


In [10]:
dali_iter = DALIGenericIterator([pipe], ['audio'])

batch shape = (19715328,)
batch shape = (23058432,)
batch shape = (21083904,)


In [11]:
for i in dali_iter:
    print(i)

[{'audio': tensor([[0., 0., 0.,  ..., 0., 0., 0.]])}]


AssertionError: Shapes do not match: DALI tensor has size [1, 23058432], but PyTorch Tensor has size [1, 19715328]

In [61]:
batch_cpu = pipe_out[0]

In [62]:
batch_cpu.at(0).shape

(8941824,)